In [1]:

"""
This is the script to simulate the nonlinear behavior of the 35 models that are calibrated on the behavior of the
reference strain, trpD9923 and the recombinant strain, trpD9923/pJLaroGfbr.
"""
import numpy as np
import pandas as pd
from configparser import ConfigParser
import time
from pytfa.io.json import load_json_model
from pytfa.optim.constraints import *

from skimpy.io.yaml import load_yaml_model
from skimpy.io.regulation import load_enzyme_regulation
from skimpy.analysis.oracle.load_pytfa_solution import load_fluxes, load_concentrations, load_equilibrium_constants
from skimpy.core.parameters import load_parameter_population
from skimpy.core.solution import ODESolutionPopulation
from skimpy.utils.tabdict import TabDict
from skimpy.utils.namespace import NET, QSSA
from skimpy.simulations.reactor import make_batch_reactor

import os
import glob
import time as T
import matplotlib.pyplot as plt
from sys import argv


''' Run parameters and paths '''
# Cellular parameters
CONCENTRATION_SCALING = 1e9  # 1 mol to 1 mmol
TIME_SCALING = 1  # 1hr
DENSITY = 1105  # g/L
GDW_GWW_RATIO = 0.3  # Assumes 70% Water
flux_scaling_factor = 1e-3 * (GDW_GWW_RATIO * DENSITY) * CONCENTRATION_SCALING / TIME_SCALING

# Ode simulation parameters
TOTAL_TIME = 60
N_STEPS = 1000


In [2]:
# parse arguments from configfile
configs = ConfigParser()
configs.read('bioreactor_configfile.ini')

base = configs['PATHS']['base']
met_model = configs['PATHS']['met_model']
model_file = configs['PATHS']['model_file']
thermo_experiment_file = configs['PATHS']['thermo_experiment_file']
regulation_file = configs['PATHS']['regulation_file']
kinetic_params_file = configs['PATHS']['kinetic_params_file']
steady_states_file = configs['PATHS']['steady_states_file']
generated_parameter_name_file = configs['PATHS']['generated_parameter_names']
output_folder = configs['BIOREACTOR_SIMULATION']['output_dir']
steady_states_sample = int(configs['BIOREACTOR_SIMULATION']['steady_states_sample'])


path_to_kmodel = f'{base}/{met_model}/kinetic/{model_file}'
path_to_tmodel = f'{base}/{met_model}/thermo/{thermo_experiment_file}'
path_to_params = f'{base}/{met_model}/parameters/{kinetic_params_file}'
path_to_regulation_data = f'{base}/{met_model}/{regulation_file}'
path_to_tfa_samples = f'{base}/{met_model}/steady_state_samples/{steady_states_file}'


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:

'''
Model and simulation initiation + preparation
'''

# Load models
tmodel = load_json_model(path_to_tmodel)
kmodel_draft = load_yaml_model(path_to_kmodel)

# Add regulation data to kinetic model
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel_draft.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel_draft.reactants.keys()))]
kmodel = load_enzyme_regulation(kmodel_draft, df_regulations_all)


2024-03-11 16:43:16,457 - thermomodel_new - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2024-03-11 16:43:19,612 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [4]:
k_names = []
k_regulation = []

for k, v in kmodel.parameters.items():
    if k.startswith("km_") or "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
        if k not in k_names:
            k_names.append(k)
            
        if "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
            k_regulation.append(k)

print(f"The number of generated k in model is {len(k_names)}, \nand the number of regulation parameters is {len(k_regulation)}.")

The number of generated k in model is 464, 
and the number of regulation parameters is 106.


In [5]:
# Load set of parameters corresponding to the 35 kinetic models
parameter_population = load_parameter_population(path_to_params)

# Reactor initialization
reactor = make_batch_reactor('single_species.yaml', df_regulation= df_regulations_all)
reactor.compile_ode(add_dilution=False)
reactor_volume = reactor.models.strain_1.parameters.strain_1_volume_e.value

kinetic_models = list(parameter_population._index.keys())

2024-03-11 16:43:24,393 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [6]:
sols_wt= []
for this_model in kinetic_models:
    print("Running model: {}".format(this_model))
    # Get TFA and kinetic model indices
    tfa_ix = steady_states_sample

    # Load tfa sample and kinetic parameters into kinetic model
    tfa_sample = pd.read_csv(path_to_tfa_samples, header=0, index_col=0).iloc[tfa_ix]
    parameter_set = parameter_population[this_model]
    kmodel.parameters = parameter_set

    # Load steady-state fluxes and concentrations into the scaffold kmodel
    flux_series = load_fluxes(tfa_sample, tmodel, kmodel,
                         density=DENSITY,
                         ratio_gdw_gww=GDW_GWW_RATIO,
                         concentration_scaling=CONCENTRATION_SCALING,
                         time_scaling=TIME_SCALING)
    
    fluxes = pd.Series([flux_series[reaction.name] for reaction in kmodel.reactions.values()])

    concentrations = load_concentrations(tfa_sample, tmodel, kmodel,
                                         concentration_scaling=CONCENTRATION_SCALING)


    # Fetch equilibrium constants
    load_equilibrium_constants(tfa_sample, tmodel, kmodel,
                               concentration_scaling=CONCENTRATION_SCALING,
                               in_place=True)
    
    def reset_reactor():
        """
        Function to reset the reactor and load the concentrations and parameters before each simulation
        """
        # Parameterize the rector and initialize with incolum and medium data
        reactor.parametrize(parameter_set, 'strain_1')
        reactor.initialize(concentrations, 'strain_1')
        reactor.initial_conditions['biomass_strain_1'] = 0.037 * 0.05 / 0.28e-12

        for met_ in reactor.medium.keys():
            LC_id = 'LC_' + met_
            LC_id = LC_id.replace('_L', '-L')
            LC_id = LC_id.replace('_D', '-D')
            reactor.initial_conditions[met_] = np.exp(tfa_sample.loc[LC_id]) * 1e9

        # Volume parameters for the reactor
        reactor.models.strain_1.parameters.strain_1_volume_e.value = reactor_volume
        reactor.models.strain_1.parameters.strain_1_cell_volume_e.value = 1.0  # 1.0 #(mum**3)
        reactor.models.strain_1.parameters.strain_1_cell_volume_c.value = 1.0  # 1.0 #(mum**3)
        reactor.models.strain_1.parameters.strain_1_cell_volume_p.value = 1.0  # 1.0 #(mum**3)
        reactor.models.strain_1.parameters.strain_1_volume_c.value = 0.9 * 1.0  # (mum**3)
        reactor.models.strain_1.parameters.strain_1_volume_p.value = 0.1 * 1.0  # (mum**3)

    ''' 1. Simulation of trpD9923'''
    reset_reactor()
    import time
    start = T.time()
    print("Starting Simulation")
    if hasattr(reactor, 'solver'):
        delattr(reactor, 'solver')

    # Function to stop integration
    def rootfn(t, y, g, user_data):
        t_0 = user_data['time_0']
        t_max = user_data['max_time']
        curr_t = time.time()
        if (curr_t - t_0) >= t_max:
            g[0] = 0
            print('Did not converge in time')
        else:
            g[0] = 1


    t0 = time.time()
    user_data = {'time_0': t0,
                 'max_time': 60} #2 minutes in seconds

    # Solve the ODEs
    sol_ode_wildtype = reactor.solve_ode(np.linspace(0, TOTAL_TIME, N_STEPS),
        solver_type='cvode',
        rtol=1e-9,
        atol=1e-9,
        max_steps=1e9,
        rootfn=rootfn,
        nr_rootfns=1,
        user_data=user_data)
    end = T.time()
    print("WT: {}".format(end - start))

    final_biomass = sol_ode_wildtype.concentrations.iloc[-1]['biomass_strain_1'] * 0.28e-12 / 0.05
    final_anthranilate = sol_ode_wildtype.concentrations.iloc[-1]['anth_e'] * 1e-6 * 136.13
    final_glucose = sol_ode_wildtype.concentrations.iloc[-1]['glc_D_e'] * 1e-6 * 180.156
    print("Final biomass is : {}, final anthranilate is : {}, final glucose is : {}".format(final_biomass,
                                                                                                final_anthranilate,
                                                                                                final_glucose))
    sols_wt.append(sol_ode_wildtype)


# Store all ode solutions
solpop = ODESolutionPopulation(sols_wt, kinetic_models)
solpop.data.to_csv(output_folder + 'ode_sols_wt.csv')


Running model: 0
Starting Simulation
Did not converge in time
WT: 60.030884742736816
Final biomass is : 1.1217708680097498, final anthranilate is : 14.849202119220099, final glucose is : 7.633845706451154e-05
Running model: 1
Starting Simulation
Did not converge in time
WT: 60.013331174850464
Final biomass is : 1.17990893926472, final anthranilate is : 13.791897908793267, final glucose is : 2798.954102770763
Running model: 2
Starting Simulation
WT: 40.394294023513794
Final biomass is : 1.301868262975232, final anthranilate is : 14.813917469395788, final glucose is : 2608.525963647406
Running model: 3
Starting Simulation
WT: 11.128847599029541
Final biomass is : 0.23386761505456993, final anthranilate is : 7.493597440899382, final glucose is : 5665.621361231679
Running model: 4
Starting Simulation
Did not converge in time
WT: 60.01104760169983
Final biomass is : 0.22477683815187283, final anthranilate is : 7.471192734797642, final glucose is : 3961.2868247375104
Running model: 5
Startin

In [7]:
sols_wt[1].concentrations

,biomass_strain_1,acald_e,ac_e,akg_e,anth_e,co2_e,lac_D_e,etoh_e,for_e,glc_D_e,...,strain_1_ser_L_p,strain_1_skm_c,strain_1__3dhsk_c,strain_1_skm5p_c,strain_1_succ_p,strain_1_r5p_c,strain_1_trp_L_p,strain_1__34hpp_c,strain_1_tyr_L_p,strain_1_pphn_c
0,6.607143e+09,9.936514,1.000000e+01,0.191176,10566.458886,100.116513,10.000000,1917.915556,1.754362e+06,5.475674e+07,...,244.081419,35880.198908,436383.752747,37766.097906,4064.492772,7.803086e+05,15419.867896,5.000000e+05,103.831589,101.132465
1,6.731465e+09,10.336290,6.746646e+02,1.387527,10938.146067,100.116513,11.409989,1919.041170,1.754363e+06,5.474612e+07,...,244.780218,35415.421540,426452.235657,37327.066567,4067.394940,7.811023e+05,15412.753018,5.062060e+05,104.092136,103.639841
2,6.858363e+09,10.395992,1.353101e+03,2.607037,11309.148792,100.116513,12.848526,1920.188142,1.754364e+06,5.473530e+07,...,245.491586,32725.125835,371061.962531,34815.633827,4073.619174,7.842516e+05,15405.469225,5.175936e+05,104.346273,106.090580
3,6.987969e+09,10.444118,2.046011e+03,3.851204,11674.020851,100.116513,14.316951,1921.357910,1.754365e+06,5.472427e+07,...,246.324726,28860.821158,299182.416212,31175.901771,4077.986884,7.890486e+05,15398.034218,5.255921e+05,104.543966,107.285310
4,7.120259e+09,10.489990,2.753275e+03,5.121218,12031.660769,100.116513,15.814750,1922.551436,1.754364e+06,5.471304e+07,...,247.393949,26132.379181,253955.256867,28558.668247,4080.073230,7.937868e+05,15390.458170,5.301262e+05,104.663895,108.199613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,2.106980e+11,169.991061,1.091246e+06,154460.603526,101314.166852,100.116513,1768.138594,3518.063579,4.287941e+05,1.682445e+07,...,135230.792785,75.630145,13027.370423,0.081567,6.114027,2.678532e+08,3787.367787,1.669931e+07,2094.775649,128.104390
238,2.106980e+11,169.398642,1.091246e+06,158319.797871,101314.167499,100.116513,1768.139905,3518.063757,4.187321e+05,1.650009e+07,...,137368.018571,74.532377,12879.719071,0.079587,5.997689,2.736373e+08,3787.364504,1.675715e+07,2053.135252,127.288535
239,2.106980e+11,168.810447,1.091246e+06,162195.691045,101314.168172,100.116513,1768.141173,3518.063939,4.089062e+05,1.617726e+07,...,139498.080381,73.508732,12746.200719,0.077686,5.883631,2.794155e+08,3787.361203,1.681448e+07,2011.844939,126.554925
240,2.106980e+11,168.226406,1.091245e+06,166087.676961,101314.168875,100.116513,1768.142400,3518.064125,3.993109e+05,1.585599e+07,...,141621.062115,72.551263,12625.347134,0.075857,5.771795,2.851864e+08,3787.357883,1.687133e+07,1970.919434,125.894193
